In [34]:
import os
import shutil
import hashlib
from PIL import Image
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import random
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras import layers, models
from tensorflow.keras.layers import BatchNormalization
#import openai
#import json


In [35]:

# ----------------------------
# Configuration
# ----------------------------
SOURCE_DIR = "C:/Users/Ayush/ath_codee/2_custom_plant_disease_dataset__"            # path to your raw dataset
CLEANED_DIR = "C:/Users/Ayush/ath_codee/new_custom_cleaned_dataset"   # output folder for cleaned dataset
IMG_SIZE = (224, 224)                    # target size for CNN
os.makedirs(CLEANED_DIR, exist_ok=True)

# ----------------------------
# 1️⃣ Remove Corrupted Images
# ----------------------------
def remove_corrupted_images(source_dir):
    removed = 0
    for folder in os.listdir(source_dir):
        folder_path = os.path.join(source_dir, folder)
        if not os.path.isdir(folder_path):
            continue
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            try:
                img = Image.open(file_path)
                img.verify()  # verifies if image is readable
            except Exception:
                print(f"❌ Removing corrupted image: {file_path}")
                os.remove(file_path)
                removed += 1
    print(f"✅ Removed {removed} corrupted images.")

remove_corrupted_images(SOURCE_DIR)

# ----------------------------
# 2️⃣ Remove Duplicate Images
# ----------------------------
def remove_duplicate_images(source_dir):
    seen_hashes = set()
    duplicates = 0
    for folder in os.listdir(source_dir):
        folder_path = os.path.join(source_dir, folder)
        if not os.path.isdir(folder_path):
            continue
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            with open(file_path, "rb") as f:
                file_hash = hashlib.md5(f.read()).hexdigest()
            if file_hash in seen_hashes:
                print(f"🗑 Removing duplicate image: {file_path}")
                os.remove(file_path)
                duplicates += 1
            else:
                seen_hashes.add(file_hash)
    print(f"✅ Removed {duplicates} duplicate images.")

remove_duplicate_images(SOURCE_DIR)

# ----------------------------
# 3️⃣ Resize Images
# ----------------------------
def resize_images(source_dir, output_dir, target_size=(224, 224)):
    for folder in os.listdir(source_dir):
        folder_path = os.path.join(source_dir, folder)
        if not os.path.isdir(folder_path):
            continue
        output_folder = os.path.join(output_dir, folder)
        os.makedirs(output_folder, exist_ok=True)
        for file in os.listdir(folder_path):
            src_path = os.path.join(folder_path, file)
            dst_path = os.path.join(output_folder, file)
            try:
                img = Image.open(src_path).convert("RGB")
                img = img.resize(target_size)
                img.save(dst_path)
            except Exception as e:
                print(f"⚠️ Error resizing {file}: {e}")
    print(f"✅ All images resized to {target_size} and saved in '{output_dir}'.")

resize_images(SOURCE_DIR, CLEANED_DIR, IMG_SIZE)

# ----------------------------
# 4️⃣ Data Normalization (for model input)
# ----------------------------
# Note: We'll apply normalization dynamically using ImageDataGenerator below.
# This step ensures pixel values are scaled between 0 and 1.

datagen = ImageDataGenerator(rescale=1./255)
print("✅ Data normalization (rescaling 1./255) will be applied during training.")

# ----------------------------
# 5️⃣ Data Augmentation
# ----------------------------
# You’ll apply this dynamically during training for better generalization.
# Example augmentations for plant leaf datasets:

aug_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=25,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest',
    validation_split=0.15  # 85% training, 15% validation
)

# Example usage — generator creation
train_gen = aug_datagen.flow_from_directory(
    CLEANED_DIR,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

val_gen = aug_datagen.flow_from_directory(
    CLEANED_DIR,
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

print("✅ Data augmentation pipeline ready (rotation, flip, zoom, brightness, etc.)")


✅ Removed 0 corrupted images.
✅ Removed 0 duplicate images.
✅ All images resized to (224, 224) and saved in 'C:/Users/Ayush/ath_codee/new_custom_cleaned_dataset'.
✅ Data normalization (rescaling 1./255) will be applied during training.
Found 72 images belonging to 27 classes.
Found 0 images belonging to 27 classes.
✅ Data augmentation pipeline ready (rotation, flip, zoom, brightness, etc.)


In [36]:

# Paths
SOURCE_DIR = "C:/Users/Ayush/ath_codee/2_custom_plant_disease_dataset__"
DEST_DIR = "C:/Users/Ayush/ath_codee/new_custom_split_dataset"

# Ratios
TRAIN_SPLIT = 0.8
VAL_SPLIT = 0.1
TEST_SPLIT = 0.1

# Create destination folders
for split in ["train", "val", "test"]:
    os.makedirs(os.path.join(DEST_DIR, split), exist_ok=True)

# Split each class folder
for class_name in os.listdir(SOURCE_DIR):
    class_path = os.path.join(SOURCE_DIR, class_name)
    if not os.path.isdir(class_path):
        continue
    
    # Destination class folders
    train_folder = os.path.join(DEST_DIR, "train", class_name)
    val_folder = os.path.join(DEST_DIR, "val", class_name)
    test_folder = os.path.join(DEST_DIR, "test", class_name)
    os.makedirs(train_folder, exist_ok=True)
    os.makedirs(val_folder, exist_ok=True)
    os.makedirs(test_folder, exist_ok=True)
    
    # Get image files
    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    random.shuffle(images)

    # Calculate split indices
    total = len(images)
    train_end = int(total * TRAIN_SPLIT)
    val_end = train_end + int(total * VAL_SPLIT)

    # Split lists
    train_imgs = images[:train_end]
    val_imgs = images[train_end:val_end]
    test_imgs = images[val_end:]

    # Copy images
    for img in train_imgs:
        shutil.copy(os.path.join(class_path, img), os.path.join(train_folder, img))
    for img in val_imgs:
        shutil.copy(os.path.join(class_path, img), os.path.join(val_folder, img))
    for img in test_imgs:
        shutil.copy(os.path.join(class_path, img), os.path.join(test_folder, img))
    
    print(f"✅ {class_name}: {len(train_imgs)} train, {len(val_imgs)} val, {len(test_imgs)} test")

print("\n🎯 Data successfully split and saved to 'custom_split_dataset/'")


✅ Aloe Leaf spot: 2 train, 0 val, 1 test
✅ Aloe Soft rot: 1 train, 0 val, 1 test
✅ Arali Leaf blight: 2 train, 0 val, 1 test
✅ Bamboo Rust: 1 train, 0 val, 1 test
✅ banana Panama wilt: 1 train, 0 val, 1 test
✅ banana Sigatoka: 2 train, 0 val, 1 test
✅ Chilli Leaf curl virus: 1 train, 0 val, 1 test
✅ Curry leaf black spot: 1 train, 0 val, 1 test
✅ Ginger Leaf spot: 2 train, 0 val, 1 test
✅ Guava Anthracnose: 0 train, 0 val, 0 test
✅ Guava Rust: 2 train, 0 val, 1 test
✅ Hibiscus Leaf spot: 1 train, 0 val, 1 test
✅ Hibiscus Rust: 3 train, 0 val, 1 test
✅ Jasmine Leaf spot: 1 train, 0 val, 1 test
✅ Jasmine Rust: 2 train, 0 val, 1 test
✅ Lemon grass Helminthosporium spot: 2 train, 0 val, 1 test
✅ Lemon grass Leaf blight: 2 train, 0 val, 1 test
✅ Lemon grass Rust: 1 train, 0 val, 1 test
✅ Mint Leaf Blight: 2 train, 0 val, 1 test
✅ Mint Rust: 3 train, 0 val, 1 test
✅ Neem Powdery mildew: 0 train, 0 val, 1 test
✅ Pepper Phytophthora foot rot: 1 train, 0 val, 1 test
✅ Powdery mildew: 4 train, 0

In [ ]:
train_dir = "C:/Users/Ayush/ath_codee/them...........DATASETS!!/2_custom_cleaned_dataset/custom_split_dataset/train"
val_dir = "C:/Users/Ayush/ath_codee/them...........DATASETS!!/2_custom_cleaned_dataset/custom_split_dataset/val"
test_dir = "C:/Users/Ayush/ath_codee/them...........DATASETS!!/2_custom_cleaned_dataset/custom_split_dataset/test"


In [37]:
train_gen = aug_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_gen = aug_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_gen = datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 46 images belonging to 27 classes.
Found 0 images belonging to 27 classes.
Found 26 images belonging to 27 classes.


In [38]:
import os
import shutil
import random

def split_dataset(
    source_dir, train_dir, val_dir, test_dir,
    train_split=0.7, val_split=0.15, test_split=0.15,
    seed=42
):
    random.seed(seed)
    # Create directories if not exists
    for d in [train_dir, val_dir, test_dir]:
        if not os.path.exists(d):
            os.makedirs(d)

    classes = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]

    for cls in classes:
        cls_path = os.path.join(source_dir, cls)
        images = [f for f in os.listdir(cls_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        random.shuffle(images)

        n_total = len(images)
        n_train = int(n_total * train_split)
        n_val = int(n_total * val_split)
        n_test = n_total - n_train - n_val  # rest

        # Make class-wise directories in train/val/test
        for split_dir in [train_dir, val_dir, test_dir]:
            os.makedirs(os.path.join(split_dir, cls), exist_ok=True)

        # Copy files to respective dirs
        for i, img in enumerate(images):
            src_path = os.path.join(cls_path, img)
            if i < n_train:
                dst_path = os.path.join(train_dir, cls, img)
            elif i < (n_train + n_val):
                dst_path = os.path.join(val_dir, cls, img)
            else:
                dst_path = os.path.join(test_dir, cls, img)
            shutil.copy2(src_path, dst_path)

    print(f"Dataset split complete:")
    print(f"Train dir: {train_dir}")
    print(f"Validation dir: {val_dir}")
    print(f"Test dir: {test_dir}")

# Example usage:
source_dataset_dir = 'C:/Users/Ayush/ath_codee/2_custom_plant_disease_dataset__'
train_directory = 'path_to_train_dir'
val_directory = 'path_to_val_dir'
test_directory = 'path_to_test_dir'

split_dataset(source_dataset_dir, train_directory, val_directory, test_directory)


Dataset split complete:
Train dir: path_to_train_dir
Validation dir: path_to_val_dir
Test dir: path_to_test_dir


In [10]:

# ----------------------------
# 1️⃣ Load the Pretrained Model
# ----------------------------
# Path to your pretrained model
MODEL_PATH = "C:/Users/Ayush/ath_codee/1_CNN_MODEL_code/efficientnetb0_plant_species.h5"  # or plant_classifier_dataframe.h5
model = load_model(MODEL_PATH)
print("Loaded pretrained CNN model successfully!")

# ----------------------------
# 2️⃣ Prepare Custom Dataset Paths
# ----------------------------
train_dir = "C:/Users/Ayush/ath_codee/new_custom_split_dataset/train"
val_dir = "C:/Users/Ayush/ath_codee/new_custom_split_dataset/val"
test_dir = "C:/Users/Ayush/ath_codee/new_custom_split_dataset/test"

IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# ----------------------------
# 3️⃣ Set Up Data Generators
# ----------------------------
# Augmentation + normalization
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=25,
    width_shift_range=0.15,
    height_shift_range=0.15,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2]
)

val_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

test_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

# Generators
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# ----------------------------
# 4️⃣ Adjust the Model Output Layer
# ----------------------------
# The old model was trained on N classes (medicinal species)
# The new dataset has possibly different class count

num_new_classes = len(train_gen.class_indices)
print(f"🌿 Custom dataset has {num_new_classes} classes.")

# Remove old top layers and add new classification head
base_model = model.layers[0]  # EfficientNet base
base_model.trainable = False  # freeze for initial training

new_model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),
    layers.Dense(num_new_classes, activation='softmax')
])

# ----------------------------
# 5️⃣ Compile Model
# ----------------------------
new_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# ----------------------------
# 6️⃣ Train the Model (Transfer Learning Phase)
# ----------------------------
callbacks = [
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, verbose=1),
    ModelCheckpoint("fine_tuned_model_stage1.h5", save_best_only=True)
]

history_stage1 = new_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=callbacks
)

# ----------------------------
# 7️⃣ Fine-Tune Stage (Unfreeze Top Layers)
# ----------------------------
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

new_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

callbacks_stage2 = [
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, verbose=1),
    ModelCheckpoint("fine_tuned_model_final.h5", save_best_only=True)
]

history_stage2 = new_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=callbacks_stage2
)

# ----------------------------
# 8️⃣ Evaluate on Test Set
# ----------------------------
test_loss, test_acc = new_model.evaluate(test_gen)
print(f" Final Model Test Accuracy: {test_acc * 100:.2f}%")

# ----------------------------
# 9️⃣ Save Final Model
# ----------------------------
new_model.save("custom_plant_disease_classifier.h5")
print("Fine-tuned custom disease model saved as 'New_custom_plant_disease_classifier.h5'")


Loaded pretrained CNN model successfully!
Found 46 images belonging to 27 classes.
Found 0 images belonging to 27 classes.
Found 26 images belonging to 27 classes.
🌿 Custom dataset has 27 classes.


NameError: name 'models' is not defined

In [ ]:

# -----------------------------
# 1️⃣ Setup OpenAI API key
# -----------------------------
# Make sure to set your API key as environment variable before running:
# export OPENAI_API_KEY="your_api_key_here"
openai.api_key = os.getenv("OPENAI_API_KEY")

# -----------------------------
# 2️⃣ Load CNN Model & Labels
# -----------------------------
model = load_model("C:/Users/Ayush/ath_codee/1_CNN_MODEL_code/custom_plant_disease_classifier.h5")

# -----------------------------
# 3️⃣ Preprocess and Predict
# -----------------------------
def preprocess_image(img_path, target_size=(224, 224)):
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"❌ Image not found: {img_path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    img = img.astype("float32") / 255.0
    img = np.expand_dims(img, axis=0)
    return img

def predict_plant_disease(img_path):
    img = preprocess_image(img_path)
    preds = model.predict(img)
    pred_idx = np.argmax(preds)
    confidence = np.max(preds) * 100
    label = class_names[pred_idx] if class_names else f"Class_{pred_idx}"
    return label, confidence

# -----------------------------
# 4️⃣ GPT Assistant Function
# -----------------------------
def get_gpt_response(pred_label, confidence):
    prompt = f"""
    You are an agricultural expert AI.
    The image analysis model detected: {pred_label} with {confidence:.2f}% confidence.

    Please provide:
    1. The name of the plant and disease
    2. A short description of this disease
    3. The main causes
    4. Organic and chemical remedies
    5. Preventive measures for farmers
    Make the tone friendly and concise.
    """

    response = openai.ChatCompletion.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a helpful agricultural assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7,
        max_tokens=400
    )
    return response.choices[0].message['content'].strip()

# -----------------------------
# 5️⃣ Full AI Assistant Flow
# -----------------------------
def run_plant_ai_assistant(image_path):
    print("🧠 Analyzing image...")
    pred_label, conf = predict_plant_disease(image_path)
    print(f"✅ Predicted: {pred_label} ({conf:.2f}% confidence)")
    print("💬 Asking GPT-4.0 for expert advice...\n")

    advice = get_gpt_response(pred_label, conf)
    print("🌿 GPT-4.0 Plant Disease Assistant says:\n")
    print(advice)

# -----------------------------
# 6️⃣ Example Run
# -----------------------------
image_path = "test_images/leaf.jpg"  # change to your image
run_plant_ai_assistant(image_path)


In [40]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)


TensorFlow version: 2.20.0


In [41]:
import keras
print(keras.__version__)


3.12.0


In [ ]:
OLD_MODEL_PATH = "C:/Users/Ayush/ath_codee/1_CNN_MODEL_code/efficientnetb0_plant_species.h5"

base_model = load_model(OLD_MODEL_PATH)

# Inspect old model summary (optional)
base_model.summary()

In [9]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Constants
DATASET_DIR = "C:/Users/Ayush/ath_codee/new_custom_cleaned_dataset"
OLD_MODEL_PATH = "efficientnetb0_plant_species.h5"
NEW_MODEL_PATH = "efficientnetb0_plant_species_finetuned.h5"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

# Load old model WITHOUT top layer for architecture adjustment
base_model = load_model(OLD_MODEL_PATH)

# Inspect old model summary (optional)
base_model.summary()

# Extract existing layers except the last Dense layer output
x = base_model.layers[-2].output  # Assuming last layer is Dense classifier

# Create DataFrame of filenames and corresponding labels from cleaned dataset
images = []
labels = []

for folder in os.listdir(DATASET_DIR):
    folder_path = os.path.join(DATASET_DIR, folder)
    if not os.path.isdir(folder_path):
        continue
    for fname in os.listdir(folder_path):
        if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
            images.append(os.path.join(folder_path, fname))
            labels.append(folder)

df = pd.DataFrame({'filename': images, 'label': labels})

# Filter classes with at least two samples for stratify splitting
counts = df['label'].value_counts()
valid_labels = counts[counts >= 2].index
df = df[df['label'].isin(valid_labels)]

# Train/Test split stratified by label
num_classes = len(df['label'].unique())
total_samples = len(df)
test_size = max(0.2, num_classes / total_samples)  # dynamic test size based on num classes

train_df, test_df = train_test_split(
    df,
    test_size=test_size,
    stratify=df['label'],
    random_state=42
)

# Data generators with preprocessing
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)
test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename', y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='filename', y_col='label',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

num_classes = len(train_generator.class_indices)
# Assuming `x` is the last layer before output:
new_outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
model = tf.keras.Model(inputs=base_model.input, outputs=new_outputs)

# Adjust output layer to match new number of classes
new_outputs = Dense(num_classes, activation='softmax')(x)

# Construct new Model with old input and new output
model = Model(inputs=base_model.input, outputs=new_outputs)

# Optionally freeze base layers for initial training
for layer in base_model.layers[:-1]:
    layer.trainable = False


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 80)             │       102,480 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,152,053 (15.84 MB)

 Trainable params: 1,453,440 (5.54 MB)

 Non-trainable params: 2,698,611 (10.29 MB)

 Optimizer params: 2 (12.00 B)

Found 46 validated image filenames belonging to 25 classes.
Found 25 validated image filenames belonging to 24 classes.


In [ ]:
print("Train generator class indices:", train_generator.class_indices)
print("Number of classes in train generator:", len(train_generator.class_indices))
print("Model output layer units:", model.output_shape[-1])


In [3]:
import os
import shutil
import hashlib
from PIL import Image
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# Constants
DATASET_PATH = "C:/Users/Ayush/ath_codee/2_custom_plant_disease_dataset__"
CLEANED_DATASET_PATH = "C:/Users/Ayush/ath_codee/cleaned_dataset_3"
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

# Step 1: Remove corrupted images and copy valid images to cleaned_dataset
def remove_corrupted_and_copy(src_path, dest_path):
    os.makedirs(dest_path, exist_ok=True)
    for class_folder in os.listdir(src_path):
        class_path = os.path.join(src_path, class_folder)
        if not os.path.isdir(class_path):
            continue
        dest_class_path = os.path.join(dest_path, class_folder)
        os.makedirs(dest_class_path, exist_ok=True)
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            try:
                img = Image.open(img_path)
                img.verify()  # Verify image valid
                # Reopen and convert to RGB then save resized copy
                img = Image.open(img_path).convert("RGB")
                img = img.resize(IMAGE_SIZE)
                img.save(os.path.join(dest_class_path, img_file))
            except (IOError, SyntaxError) as e:
                print(f"Removing corrupted image: {img_path}")

# Step 2: Remove duplicate images (by hash) in cleaned_dataset
def remove_duplicates(dataset_path):
    hashes = set()
    for class_folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, class_folder)
        if not os.path.isdir(folder_path):
            continue
        for img_file in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_file)
            with open(img_path, "rb") as f:
                file_hash = hashlib.md5(f.read()).hexdigest()
            if file_hash in hashes:
                print(f"Duplicate found and removed: {img_path}")
                os.remove(img_path)
            else:
                hashes.add(file_hash)

# Step 6: Check class imbalance and output class counts
def print_class_distribution(dataset_path):
    class_counts = {}
    for class_folder in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_folder)
        if os.path.isdir(class_path):
            count = len([f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
            class_counts[class_folder] = count
    print("Class distribution after cleaning:")
    for cls, cnt in class_counts.items():
        print(f"{cls}: {cnt}")

# Run cleaning steps
remove_corrupted_and_copy(DATASET_PATH, CLEANED_DATASET_PATH)
remove_duplicates(CLEANED_DATASET_PATH)
print_class_distribution(CLEANED_DATASET_PATH)

# Step 4 & 5: Data normalization and augmentation with ImageDataGenerator
train_datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    CLEANED_DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

validation_generator = train_datagen.flow_from_directory(
    CLEANED_DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=42
)

# The train_generator and validation_generator now have normalized and augmented batches ready for model training.

print(f"Number of classes: {len(train_generator.class_indices)}")
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {validation_generator.samples}")


Duplicate found and removed: C:/Users/Ayush/ath_codee/cleaned_dataset_3\Lemon grass Rust\3.jpg
Class distribution after cleaning:
Aloe Leaf spot: 3
Aloe Soft rot: 2
Arali Leaf blight: 3
Bamboo Rust: 2
banana Panama wilt: 2
banana Sigatoka: 3
Chilli Leaf curl virus: 2
Curry leaf black spot: 2
Ginger Leaf spot: 3
Guava Anthracnose: 0
Guava Rust: 3
Hibiscus Leaf spot: 2
Hibiscus Rust: 4
Jasmine Leaf spot: 2
Jasmine Rust: 3
Lemon grass Helminthosporium spot: 3
Lemon grass Leaf blight: 3
Lemon grass Rust: 2
Mint Leaf Blight: 3
Mint Rust: 4
Neem Powdery mildew: 1
Pepper Phytophthora foot rot: 2
Powdery mildew: 5
Rose Black spot: 2
Sooty mold: 4
Tumeric Leaf blotch: 4
Tumeric Leaf spot: 3
Found 71 images belonging to 27 classes.
Found 1 images belonging to 27 classes.
Number of classes: 27
Training samples: 71
Validation samples: 1


In [4]:
import os
import shutil
import random

# Constants
DATASET_DIR = 'C:/Users/Ayush/ath_codee/cleaned_dataset_3'  # Path to your cleaned dataset folder
TRAIN_DIR = 'C:/Users/Ayush/ath_codee/cleaned_dataset_3_train'
TEST_DIR = 'C:/Users/Ayush/ath_codee/cleaned_dataset_3_test'
TEST_SIZE = 0.15  # 15% for testing

# Create directories if they don't exist
os.makedirs(TRAIN_DIR, exist_ok=True)
os.makedirs(TEST_DIR, exist_ok=True)

# For each class folder in the dataset
for class_folder in os.listdir(DATASET_DIR):
    class_path = os.path.join(DATASET_DIR, class_folder)
    if not os.path.isdir(class_path):
        continue

    images = os.listdir(class_path)
    random.shuffle(images)

    # Calculate split index
    split_idx = int(len(images) * (1 - TEST_SIZE))
    train_images = images[:split_idx]
    test_images = images[split_idx:]

    # Create class folders in train and test directories
    train_class_path = os.path.join(TRAIN_DIR, class_folder)
    test_class_path = os.path.join(TEST_DIR, class_folder)
    os.makedirs(train_class_path, exist_ok=True)
    os.makedirs(test_class_path, exist_ok=True)

    # Move images to train folder
    for img in train_images:
        shutil.copy(os.path.join(class_path, img), train_class_path)

    # Move images to test folder
    for img in test_images:
        shutil.copy(os.path.join(class_path, img), test_class_path)

print("Train-test split completed and data saved.")


Train-test split completed and data saved.


In [5]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Set dataset path and image parameters
DATASET_PATH = 'C:/Users/Ayush/ath_codee/cleaned_dataset_3'
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 20

# Data augmentation and preprocessing with EfficientNet preprocessing
datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    validation_split=0.2
)

# Train and validation generators with categorical class_mode for multi-class classification
train_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',  # Important for one-hot encoded labels
    subset='training',
    shuffle=True,
    seed=42
)

validation_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=42
)

num_classes = len(train_generator.class_indices)  # Number of plant species + disease classes

# Load pre-trained EfficientNetB0 base model without the top classification layer
base_model = tf.keras.models.load_model('C:/Users/Ayush/ath_codee/1_CNN_MODEL_code/efficientnetb0_plant_species.h5')

# Construct new model with modified output layer for your classes
inputs = tf.keras.Input(shape=(*IMAGE_SIZE, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
outputs = Dense(num_classes, activation='softmax')(x)  # Softmax for multi-class classification

model = Model(inputs, outputs)

# Compile model with categorical crossentropy loss and accuracy metric
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Define callbacks for early stopping, learning rate reduction and best model saving
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, verbose=1),
    ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)
]

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    callbacks=callbacks
)

# Optional fine-tuning: unfreeze some layers and recompile with lowered learning rate, then train
base_model.trainable = True
fine_tune_at = len(base_model.layers) - 20  # Unfreeze last 20 layers

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

fine_tune_epochs = 10
total_epochs = EPOCHS + fine_tune_epochs

history_fine = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    callbacks=callbacks
)

print("Training and fine-tuning complete.")


Found 71 images belonging to 27 classes.
Found 1 images belonging to 27 classes.


ValueError: Input 0 of layer "global_average_pooling2d" is incompatible with the layer: expected ndim=4, found ndim=2. Full shape received: (None, 80)

In [11]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Constants
DATASET_PATH = 'C:/Users/Ayush/ath_codee/cleaned_dataset_3'
OLD_MODEL_PATH = 'C:/Users/Ayush/ath_codee/1_CNN_MODEL_code/efficientnetb0_plant_species.h5'
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)
EPOCHS = 10

# Load your existing trained model (with included pooling and classifier)
base_model = load_model(OLD_MODEL_PATH)

# Create data generators
datagen = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

train_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True,
    seed=42
)

validation_generator = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False,
    seed=42
)

# Get correct number of classes dynamically
num_classes = len(train_generator.class_indices)

# Remove last Dense layer from base_model and get second last layer output
x = base_model.layers[-3].output  # Dropout layer output (before Dense)

# Add new Dense layer for new number of classes
new_output = Dense(num_classes, activation='softmax')(x)

# Create new model
model = Model(inputs=base_model.input, outputs=new_output)

# Freeze base layers initially
for layer in model.layers[:-1]:
    layer.trainable = False

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS
)


Found 71 images belonging to 27 classes.
Found 1 images belonging to 27 classes.
Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.0563 - loss: 3.4461 - val_accuracy: 0.0000e+00 - val_loss: 3.3821
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 432ms/step - accuracy: 0.0986 - loss: 3.1252 - val_accuracy: 0.0000e+00 - val_loss: 3.1036
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 652ms/step - accuracy: 0.2817 - loss: 2.8682 - val_accuracy: 0.0000e+00 - val_loss: 3.0407
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 449ms/step - accuracy: 0.3803 - loss: 2.6699 - val_accuracy: 0.0000e+00 - val_loss: 3.1214
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 470ms/step - accuracy: 0.4648 - loss: 2.5379 - val_accuracy: 0.0000e+00 - val_loss: 2.9034
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 700ms/step - accuracy: 0.5775 - loss: 2.3322 - val_accuracy: 1.0000 - val_loss: 2.7180
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 769ms/step - accuracy: 0.6901 - loss: 2.1446 - val_accuracy: 0.0000e+00 - val_loss: 2.6041
Epoch 8/10
3/3 

In [13]:
model.save('new_efficientnetb0_disease_detector', save_format='tf')
print("Model saved in TensorFlow SavedModel format in folder 'new_efficientnetb0_disease_detector'")


ValueError: The `save_format` argument is deprecated in Keras 3. Please remove this argument and pass a file path with either `.keras` or `.h5` extension.Received: save_format=tf

In [12]:
model.save('new_efficientnetb0_disease_detector.h5')
print("Model saved as new_efficientnetb0_disease_detector.h5")


TypeError: cannot pickle 'module' object

In [14]:
model.save('new_efficientnetb0_disease_detector.keras')
print("Model saved as new_efficientnetb0_disease_detector.keras")


Model saved as new_efficientnetb0_disease_detector.keras


In [15]:
import tensorflow as tf

# Replace the path below with your actual model file path
model_path = 'new_efficientnetb0_disease_detector.keras'  # or .h5 if using legacy format

# Load the model
model = tf.keras.models.load_model(model_path)

# Print model summary
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 27)             │        34,587 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,153,334 (15.84 MB)

 Trainable params: 34,587 (135.11 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

 Optimizer params: 69,176 (270.22 KB)